In [151]:
import pandas as pd
import numpy as np

In [152]:
data_bus= pd.read_csv('E:/BusData.txt', sep="\t", header=None,names=["Data", "Address Start", "Day of week","Time Start", "Time Duration","Time Arrive"])

In [153]:
data_bus.head(20)

,Data,Address Start,Day of week,Time Start,Time Duration,Time Arrive
0,Direction. U (from univer). H (from home),NaN,NaN,NaN,NaN,NaN
1,"Day of week: Mon, Tue, Wed, Thu, Fri, Sat, Sun",NaN,NaN,NaN,NaN,NaN
2,Departure time: hhmm,NaN,NaN,NaN,NaN,NaN
3,Time take: mmss,NaN,NaN,NaN,NaN,NaN
4,Arrival time: hhmm,NaN,NaN,NaN,NaN,NaN
5,# 16.04.2018,NaN,NaN,NaN,NaN,NaN
6,U. Mon 12h05 13m55,NaN,NaN,NaN,NaN,NaN
7,H. Tue 08h36 14m19,NaN,NaN,NaN,NaN,NaN
8,U. Tue 15h05 14m14,NaN,NaN,NaN,NaN,NaN
9,H. Wed 06h41 14m48,NaN,NaN,NaN,NaN,NaN


In [154]:
data_H = data_bus['Data'].astype(str).str.startswith('H.')
data_U = data_bus['Data'].astype(str).str.startswith('U.')

In [155]:
data_clean = data_H | data_U

data = data_bus[data_clean]
data = data.reset_index(drop=True)


In [156]:
def remove_char(x):
  # print(x)
  x = str(x)
  x = x.replace('.', '')
  return x


In [157]:
data['Address Start'] = data['Data'].astype(str).str.split(" ").str[0].apply(remove_char)
data['Day of week'] = data['Data'].astype(str).str.split(" ").str[1].apply(remove_char)
data['Time Start'] = data['Data'].astype(str).str.split(" ").str[2].apply(remove_char)
data['Time Duration'] = data['Data'].astype(str).str.split(" ").str[3].apply(remove_char)
data['Time Arrive'] = data['Data'].astype(str).str.split(" ").str[4].apply(remove_char)

data = data.drop('Data', axis=1)

data.loc[134, ['Time Start', 'Time Duration']] = "06h42", "15m55"

In [158]:
def convert_data_time(x):
  x = str(x)
  if x != 'nan' and x!= '':
    if x.find('m') == -1:
      x = x + 'm00'
    
    if x.find('h') == -1:
      x = '00h' + x
    
    return x
  else:
    return None

In [159]:
data['Time Start'] = data['Time Start'].apply(convert_data_time)
data['Time Duration'] = data['Time Duration'].apply(convert_data_time)
data['Time Arrive'] = data['Time Arrive'].apply(convert_data_time)

In [160]:
import datetime

def convert_time_s(x):
  x = str(x)
  if x=='None':
    return None

  F1 = '%Hh%Mm%S'
  time_object1 = datetime.datetime.strptime(x, F1)
  dt1 = datetime.timedelta(hours=time_object1.hour,minutes=time_object1.minute, seconds=time_object1.second)

  return dt1.seconds

In [161]:
import math

def convert_time_str(x):

  if math.isnan(x):
    return None
  x = int(x)
  hours = int(x/3600)
  # print(hours)
  minutes = int((x%3600)/60)  
  # print(minutes)
  seconds = int((x%3600)%60)
  # print(seconds)
  time_string = str(hours) + 'h' + str(minutes) + 'm' + str(seconds)
  return time_string

In [162]:
import datetime

def cal_time(startTime, endTime, mode):
  string_start = str(startTime)
  string_end = str(endTime)
  F1 = '%Hh%Mm%S'
  time_object1 = datetime.datetime.strptime(string_start, F1)
  time_object2 = datetime.datetime.strptime(string_end, F1)
  dt1 = datetime.timedelta(hours=time_object1.hour,minutes=time_object1.minute, seconds=time_object1.second)
  dt2 = datetime.timedelta(hours=time_object2.hour,minutes=time_object2.minute, seconds=time_object2.second)

  if mode == 1:
    stringTime = str(dt1 + dt2)
  else:
    stringTime = str(dt2 - dt1)
  
  stringTime = stringTime.replace(':', 'h', 1)
  stringTime = stringTime.replace(':', 'm', 1)
  return stringTime

for i in range(len(data['Time Arrive'])):
#   print(i, data.loc[i, 'Time Arrive'])
  if data.loc[i, 'Time Arrive'] != None and data.loc[i, 'Time Start'] != None:
    data.loc[i, 'Time Duration'] = cal_time(data.loc[i, 'Time Start'], data.loc[i, 'Time Arrive'], 0)
# data.loc[260:280, 'Time Duration']
for i in range(len(data['Time Arrive'])):
  # print(i, data.loc[i, 'Time Arrive'])
  if data.loc[i, 'Time Arrive'] == None :
    if data.loc[i, 'Time Duration'] != None and data.loc[i, 'Time Start'] != None:
      data.loc[i, 'Time Arrive'] = cal_time(data.loc[i, 'Time Start'], data.loc[i, 'Time Duration'], 1)
# data.loc[:20,['Time Arrive']]

In [163]:
def cal_time_s(time_):
  
  # print(time_str)
  if time_ == None:
    return None
  else:
    time_str = str(time_)
    hours = int(time_str[:time_str.find('h')])
    minutes = int(time_str[time_str.find('h')+ 1:time_str.find('m')])
    seconds = int(time_str[time_str.find('m')+ 1:])
    return hours*3600 + minutes*60 + seconds

data['Time Duration'] = data['Time Duration'].apply(cal_time_s)
# data['Time Duration'].head(20)

In [164]:
data.groupby('Address Start').agg('mean')

,Time Duration
Address Start,
H,882.097744
U,893.967033


In [165]:
data_dur = data.groupby('Day of week').agg('mean')
data_dura = data_dur['Time Duration'].apply(convert_time_str)
data_dura

Day of week
Fri    0h14m37
Mon     0h14m5
Sat    0h14m35
Sun     0h14m6
Thu    0h15m15
Tue    0h14m34
Wed    0h15m21
Name: Time Duration, dtype: object

In [166]:
data['Temp Time Start'] = data['Time Start'].apply(convert_time_s)
data_ar = data.groupby(['Address Start', 'Day of week']).agg('mean')
data_ara = data_ar['Temp Time Start'].apply(convert_time_str)
data_ara

Address Start  Day of week
H              Fri               8h2m7
               Mon             7h49m48
               Sat              9h49m6
               Sun            10h32m15
               Thu              9h1m58
               Tue              8h7m40
               Wed             8h51m33
U              Fri             12h9m45
               Mon             12h2m13
               Sat            13h26m34
               Sun             13h40m0
               Thu            13h47m26
               Tue            12h30m40
               Wed             13h53m7
Name: Temp Time Start, dtype: object

In [167]:
data_y = data.groupby('Address Start').agg('mean')
data_y['Temp Time Start'].apply(convert_time_str)

Address Start
H     8h31m14
U    12h53m32
Name: Temp Time Start, dtype: object